---
# **LAB 1 - Intro to Numba**
---

# Google Colaboratory (Colab)

[Colaboratory](https://research.google.com/colaboratory/faq.html) (or Colab) is a **free research tool** from *Google* for machine learning education and research built on top of [Jupyter Notebook](https://jupyter.org/). It requires no setup and runs entirely in the **cloud**. In Google Colab you can write, execute, save and share your Jupiter Notebooks. You access powerful computing resources like **TPUs** and **GPUs** all for free through your browser. All major Python libraries like **Tensorflow**, **Scikit-learn**, **PyTorch**, **Pandas**, etc. are pre-installed. Google Colab requires no configuration, you only need a **Google Account** and then you are good to go. Your notebooks are stored in your **Google Drive**, or can be loaded from **GitHub**. Colab notebooks can be shared just as you would with Google Docs or Sheets. Simply click the Share button at the top right of any Colab notebook, or follow these Google Drive file sharing instructions.




### Shell commands

The command `uname` displays the information about the system.

* **-a option:** It prints all the system information in the following order: Kernel name, network node hostname,
kernel release date, kernel version, machine hardware name, hardware platform, operating system
.

In [ ]:
!uname -a && cat /etc/*release

### Set up Google Drive...

That snippet is used in **Google Colab** to mount your **Google Drive**.

- Imports the **Colab utility** to access Google Drive
- Mounts your Drive at the **path**:
`/content/drive/MyDrive/`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# CUDA tools...

**NVIDIA System Management Interface (nvidia-smi)**

The NVIDIA System Management Interface (**`nvidia-smi`**) is a command line utility, based on top of the NVIDIA Management Library (NVML), intended to aid in the **management** and **monitoring** of NVIDIA GPU devices.

This utility allows administrators to query GPU device state and with the appropriate privileges, permits administrators to modify GPU device state.  It is targeted at the TeslaTM, GRIDTM, QuadroTM and Titan X product, though limited support is also available on other NVIDIA GPUs.

For more details, please refer to the **`nvidia-smi`** documentation ([doc](http://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf))

For information on **Tesla T4** see:

In [ ]:
!nvidia-smi

**Numba code used as a CUDA sanity check**:
-   Imports Numba, a JIT compiler that accelerates Python code.
-	numba.cuda provides GPU (CUDA) support using NVIDIA GPUs.
	- ✔ Confirms NumPy and Numba are installed
	- ✔ Confirms CUDA drivers are visible
	- ✔ Confirms GPU compute capability
	- ✔ Helps debug environment issues before running GPU kernels

Probes the system for available CUDA-capable GPUs. Prints:
-	Number of GPUs
-	GPU names
-	Compute capability
-	Driver/runtime status


In [ ]:
import numpy as np
import numba
from numba import cuda
import warnings
warnings.filterwarnings("ignore")

print(np.__version__)
print(numba.__version__)

cuda.detect()



# ✅ Hello World!

**My first CUDA program: HelloFromGPU!**

CUDA kernel for Hello world...

In [ ]:
from numba import cuda

@cuda.jit
def hello_kernel():
    print("Hello from GPU!") # This is not printing in Colab (but it should...)

print("Starting...")
hello_kernel[1, 10]() # 1 Block, 10 Threads
cuda.synchronize()
print("...Synchronized")

# ✅ Parallel vector sum

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def add_arrays_cuda(a, b, c):
    i = cuda.grid(1) # Return the absolute position of the current thread in the entire grid of blocks (1 means to interpret grid as 1D).
    assert i < c.size
    c[i] = a[i] + b[i]

# Example
n = 1000    # size of arrays = number of threads
a = np.ones(n, dtype=np.float32) # Notice float32! (Default for GPUs - but not for Python)
b = np.ones(n, dtype=np.float32)

d_a = cuda.to_device(a) # Sends to Device (the GPU)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(a) # Allocates an array directy on GPU

# Start the n Threads. then wait for all of them to finish
add_arrays_cuda[1, n](d_a, d_b, d_c)
cuda.synchronize()

c = d_c.copy_to_host() # Copies to Host (the CPU)
print(c[:10], c[-10:])

## ↘️ TODO...

**Check Fibonacci Membership**

🔹 **Exercise Goal**

-   Given a vector $x$ of integers, build a CUDA kernel (Numba) that produces a vector $y$ such that:

$$
y_i =
\begin{cases}
1 & \text{if } x_i \text{ is a Fibonacci number} \\
0 & \text{otherwise}
\end{cases}
$$


🔹 **Input**

- `x`: 1D array of integers (e.g., `np.int32`)
- `n = x.size`

🔹 **Output**

- `y`: 1D array of `np.uint8` or `np.int32`
- Same length as `x`


🔹 **Constraints**

- One GPU thread handles one element:
  $$
  i = \text{cuda.grid}(1)
  $$
- Must include a bounds check:
  $$
  i < n
  $$
- Avoid Python objects and dynamic allocation inside the kernel


🔹 **A classic property:**

> An integer $v \ge 0$ is Fibonacci **iff** one of these is a perfect square:
$$
5v^2 + 4 \quad \text{or} \quad 5v^2 - 4
$$


🔹 **Your Tasks**

1. Write a device function:
   - `is_perfect_square(m) -> bool`

2. Write a device function:
   - `is_fib(v) -> bool`

3. Write a kernel:
   - `fib_mask(x, y)` that sets `y[i] = 1` if `x[i]` is Fibonacci else `0`

4. Write host code to:
   - allocate/copy arrays to GPU
   - launch the kernel
   - copy results back and validate

🔹 **Expected result**
-    for the first $n = 1000$ integers: [0,   1,   2,   3,   5,   8,  13,  21,  34,  55,  89, 144, 233, 377, 610, 987]

🔹 **Skeleton Code (Fill the TODOs)**

```{python}
    #| echo: true
    import numpy as np
    from numba import cuda
    import math

    @cuda.jit
    def fib_numbers(x,y):
        # TODO
        pass

    # input data
        # TODO

    # GPU memory allocation
        # TODO

    # Kernel launch
        # TODO

    # Copy back results & print indexes of Fibonacci numbers
        # TODO
```

In [ ]:
import numpy as np
from numba import cuda
import math

@cuda.jit
def is_perfect_square(m: int) -> bool:
  """Returns true if m is a perfect square"""
  sqr = math.floor(m ** 0.5)
  return sqr * sqr == m

@cuda.jit
def is_fib(v: int) -> bool:
  """Returns true if v is a Fibonacci number"""
  return is_perfect_square(5 * v * v + 4) or is_perfect_square(5 * v * v - 4)

@cuda.jit
def fib_mask(x: np.ndarray, y: np.ndarray):
  """Sets y[i] to 1 if x[i] is a Fibonacci number"""
  i = cuda.grid(1)
  if i < x.size and is_fib(x[i]): # Remember to check size
    y[i] = 1
  else:
    y[i] = 0


# input data
n = 1000
x = np.arange(0, n, dtype=np.int32)
y = np.zeros(n, dtype=np.uint8) # I could also create this directly on Device using cuda.device_array

# GPU memory allocation
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

# Kernel launch
fib_mask[1, n](d_x, d_y)

# Copy back results & print indexes of Fibonacci numbers
cuda.synchronize()
y = d_y.copy_to_host()
for num, is_fib in zip(x, y):
  if is_fib == 1:
    print(num, end=" ")
print()